Various CSV files are provided in this challenge. Before building a predictive model, it is required to preprocess each CSV file and merge them into a single CSV file. First, let us load all the provided CSV files into Panda Dataframes.

In [1]:
import re
import pandas as pd
import numpy as np
from toolz import *
from toolz.curried import *

# paths to all the csvs files provided in the challenge.
trainPath   = "/content/train.csv"
testPath    = "/content/test.csv" 
microPath   = "/content/properties.csv"
macrosPaths = ["/content/cpi.csv", "/content/interest.csv", "/content/rentIndex.csv", "/content/vacant.csv"]
geoPath     = "/content/geo_attributes.csv"

# load the csvs into pandas's Dataframe.
## load train&test_keys that contain unique identifier for each observation
train_keys = pd.read_csv(trainPath).assign(train = 1)
test_keys  = pd.read_csv(testPath).assign(train = 0).assign(price = pd.NA)
keys       = pd.concat([train_keys, test_keys])
keys       = keys.rename(columns = {"contractDate" : "date"})

## merge key with micro to borrow the date information in key.
_micro = pd.read_csv(microPath)
micro  = pd.merge(_micro, keys, on = "property_key")

## load all the macro files
macros = []
for macrosPath in macrosPaths:
    df = pd.read_csv(macrosPath)
    df = df.rename(columns = {"Data Series" : "date"})
    macros.append(df)

# load geo
geo = pd.read_csv(geoPath)

In [2]:
micro.isnull().sum()

area                0
floorRange          0
propertyType        0
district            0
typeOfArea          0
tenure              0
street              0
project             0
marketSegment       0
property_key        0
date                0
price            2331
train               0
dtype: int64

the data has no missing values beside the target variable (price) in test. So we don't need to do data imputation

Now let's define preprocessing functions for micro and macro data. For micro data we performed 4 preprocessing:

1.   encode floorRange with ordinal encoder
2.   categorizing the tenure column into binary (freehold|lease)
3.   encode typeOfArea, propertyType, marketSegment with one hot encoding
4.   make 3 new variables called day, month, and year

In [3]:
def preprocess_micro(micro):    
    """
    preprocess micro data properties.csv
    """
    def _floorRange(row):
      merged_mapped = []
      unique_merged = sorted(micro["floorRange"].unique().tolist())
      for value in micro["floorRange"]:
        index = unique_merged.index(value)
        merged_mapped.append(index)
      return merged_mapped
    
    def _tenure(row):
        """
        some tenures are too rare to be used as variables.
        Hence simply making tenure feature to 1 if freehold else 0.
        """
        tenure = row["tenure"]    
        if tenure == "Freehold" : row["tenure"] = 1
        else                    : row["tenure"] = 0        
        return row
    merged_mapped = _floorRange(micro)
    micro = (micro
             .drop(["floorRange"], axis = 1)
             .assign(floorRange = merged_mapped)
             .apply(_tenure, axis = 1))
    micro["date"] = pd.to_datetime(micro["date"])
    micro.loc[micro['typeOfArea'] == "Land", 'typeOfArea'] = 0
    micro.loc[micro['typeOfArea'] == "Strata", 'typeOfArea'] = 1
    micro['year'] = micro['date'].dt.year
    micro['month'] = micro['date'].dt.month
    micro['day'] = micro['date'].dt.day
    dummy=pd.get_dummies(micro['propertyType'])
    dummy1=pd.get_dummies(micro['marketSegment'])
    micro = pd.concat([micro, dummy, dummy1], axis = 1, join = "inner")
    micro["typeOfArea"] = micro["typeOfArea"].astype(int)
    micro = micro.drop(["propertyType", "marketSegment"], axis = 1)
    return micro

For macro data, some informations are available only in quaterly resolution, we need to re-format quaterly to monthly using simple linear interpolation.

In [4]:
def preprocess_macros(cpi, interest, rentIndex, vacant):
    """
    four files are considered as macro-related files :
        * cpi.csv
        * interest.csv
        * rentIndex.csv
        * vacant.csv
    """
    
    def yq2ym(df):
        """
        convert year-quater in string format to monthly period
        """
        
        # repeat the first row for desired interpolation result
        df = pd.concat([df.head(0), df])
        df.at[0, "date"] = "2023 1Q "
            
        df["date"] = (pd.to_datetime((df["date"]
                                      .str
                                      .replace(r"(\d+) (\d)Q ", r"\1-Q\2")))
                      .dt
                      .to_period('M'))        
        df = df.set_index("date").resample("M", convention = "end").interpolate("linear")        
        return df    

    def ym2ym(df):
        """
         convert year-month in string format to monthly period
        """
        df["date"] = (pd.to_datetime(df["date"], format = "%Y %b ")
                      .dt
                      .to_period('M'))
        return df
    
    def mergeDfs(dfs, on):        
        """
        join dataframes into one
        """
        # grab first dataframe
        all_merged = dfs[0]
        # loop through all but first data frame
        for to_merge in dfs[1:]:
            # result of merge replaces first or previously
            # merged data frame w/ all previous fields
            all_merged = pd.merge(all_merged, to_merge,
                                  how = 'inner',
                                  on  = on)
        return all_merged
    
    # apply appropriate date conversion function for each dataframe
    dfs = [ym2ym(cpi), ym2ym(interest), yq2ym(rentIndex), yq2ym(vacant)]
    
    # join all dataframs into one
    df = mergeDfs(dfs, on = "date")   
    # convert peroid[M] to datetime
    df["date"] = df["date"].dt.to_timestamp()    
    # shift month + 3 
    df["date"] = df["date"] + pd.DateOffset(months=+3)
    
    return df 

Now let's apply the two functions defined above to the micro and macro data.

In [5]:
micro = preprocess_micro(micro)
macro = preprocess_macros(*macros)

<ipython-input-4-ae9d572d4b97>:21: FutureWarning: The default value of regex will change from True to False in a future version.
  .replace(r"(\d+) (\d)Q ", r"\1-Q\2")))
<ipython-input-4-ae9d572d4b97>:21: FutureWarning: The default value of regex will change from True to False in a future version.
  .replace(r"(\d+) (\d)Q ", r"\1-Q\2")))


We merge the preprocessed micro and macro data. Also, we fill in the missing values generated from the merging process using simple mean imputation.

In [ ]:
_merged = pd.merge(micro, macro, on = "date", how = "left")
merged  = pd.merge(_merged, geo, on = ["street", "project", "district"], how = "left")
merged  = merged.fillna(merged.mean())

<ipython-input-14-1807e09f7ccd>:3: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  merged  = merged.fillna(merged.mean())
<ipython-input-14-1807e09f7ccd>:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  merged  = merged.fillna(merged.mean())


In [ ]:
train = mergeds[mergeds["train"] == 1]
test  = mergeds[mergeds["train"] == 0]

# **MODELING WITH PYCARET**

In [8]:
!pip install pycaret[full] -q

Do some set up with pycaret that define the target variable, numeric features, categorical features, and date features.

In [ ]:
from pycaret.regression import *
exp_reg101 = setup(data=train, 
                   target='price', 
                   session_id=123, 
                   numeric_features=['area', 'district', 'typeOfArea', 'tenure', 
                                     'CPI', 'InterestRate',
                                     'RentIndex', 'Available', 'Vacant', 'lat', 'lng', 'num_schools_1km',
                                     'num_supermarkets_500m', 'num_mrt_stations_500m', 'floorRange', 'year',
                                     'month', 'day', 'Apartment', 'Condominium', 'Detached',
                                     'Executive Condominium', 'Semi-detached', 'Strata Detached',
                                     'Strata Semi-detached', 'Strata Terrace', 'Terrace', 'CCR', 'OCR',
                                     'RCR'], 
                   categorical_features=["street", "project", "property_key"], 
                   date_features=["date"])

,Description,Value
0,Session id,123
1,Target,price
2,Target type,Regression
3,Original data shape,"(62949, 36)"
4,Transformed data shape,"(62949, 38)"
5,Transformed train set shape,"(44064, 38)"
6,Transformed test set shape,"(18885, 38)"
7,Numeric features,30
8,Date features,1
9,Categorical features,3


In [ ]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,171762.2644,162348945986.0302,400549.0667,0.9403,0.1119,0.0817,10.6860
et,Extra Trees Regressor,152010.0763,163088883440.8048,402141.0747,0.9401,0.1003,0.0683,21.6670
xgboost,Extreme Gradient Boosting,178061.5281,181511278592.0000,423982.3719,0.9332,0.1148,0.0828,7.5790
lightgbm,Light Gradient Boosting Machine,192273.7216,182930919416.3643,425886.7340,0.9329,0.1275,0.0955,1.3610
rf,Random Forest Regressor,167277.6694,197726167712.4150,442565.7919,0.9275,0.1089,0.0745,34.3240
gbr,Gradient Boosting Regressor,245891.4752,261524300310.3993,510422.1097,0.9041,0.1620,0.1265,9.6890
dt,Decision Tree Regressor,235409.1075,384097264652.8454,617072.6561,0.8593,0.1557,0.1048,0.8470
lasso,Lasso Regression,314034.1955,419429696319.5300,646602.8749,0.8463,0.3023,0.1729,2.7320
ridge,Ridge Regression,313891.3910,419265042533.9398,646482.8996,0.8463,0.3033,0.1728,0.5610
br,Bayesian Ridge,313945.6958,419444114333.8974,646617.7131,0.8463,0.3024,0.1728,0.4300


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [ ]:
catboost = create_model("catboost")
tuned_catboost = tune_model(catboost)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,178204.1296,216110365068.3903,464876.7203,0.9241,0.1132,0.0825
1,170838.3321,143927614578.3099,379377.9311,0.9467,0.1111,0.0822
2,171274.3047,185980748550.7526,431254.8534,0.9382,0.1129,0.0800
3,171703.8023,162332084852.7667,402904.5605,0.9425,0.1090,0.0806
4,171287.0456,175154186211.0347,418514.2605,0.9225,0.1119,0.0825
5,161644.3998,110322399203.5749,332148.1585,0.9572,0.1097,0.0805
6,169000.8583,116884769004.1683,341884.1456,0.9603,0.1090,0.0817
7,174931.0744,136021045996.6553,368810.3117,0.9471,0.1123,0.0832
8,174459.3076,160858820407.6258,401072.0888,0.9398,0.1185,0.0839


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,192345.3457,236167784565.9542,485970.9709,0.9170,0.1208,0.0900
1,183263.8250,162520939876.8749,403138.8593,0.9398,0.1185,0.0889
2,180377.5845,200988296211.7058,448317.1826,0.9332,0.1185,0.0848
3,180412.2316,190649841481.3800,436634.6774,0.9325,0.1140,0.0859
4,182209.3682,175652306850.0113,419108.9439,0.9223,0.1186,0.0881
5,174012.2815,116021938433.8438,340619.9325,0.9550,0.1176,0.0880
6,185251.3630,143972766389.7194,379437.4341,0.9511,0.1203,0.0914
7,184299.0990,144588104205.0522,380247.4250,0.9438,0.1185,0.0888
8,185754.4203,185807641481.5186,431054.1051,0.9304,0.1239,0.0895


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
xgboost = create_model("xgboost")
lightgbm = create_model("lightgbm")
tuned_xgboost = tune_model(xgboost)
tuned_lightgbm = tune_model(lightgbm)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,185341.5000,213245067264.0000,461784.6562,0.9251,0.1175,0.0858
1,178394.0625,154918944768.0000,393597.4375,0.9426,0.1142,0.0836
2,180954.5312,216079974400.0000,464844.0312,0.9282,0.1168,0.0814
3,179640.2344,184172150784.0000,429152.8438,0.9348,0.1128,0.0826
4,176037.9375,210283560960.0000,458566.8438,0.9070,0.1154,0.0836
5,166728.5156,118588268544.0000,344366.4688,0.9540,0.1119,0.0806
6,180029.0938,167187218432.0000,408885.3438,0.9432,0.1124,0.0826
7,178729.3438,156548628480.0000,395662.2812,0.9392,0.1131,0.0832
8,175300.5938,156215525376.0000,395241.0938,0.9415,0.1191,0.0836


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,198501.1507,222044560588.8475,471216.0445,0.9220,0.1274,0.0965
1,189154.3619,156015537063.5287,394988.0214,0.9422,0.1259,0.0952
2,191036.5637,218481607378.6800,467420.1615,0.9274,0.1275,0.0924
3,190691.1966,179630451872.8750,423828.3283,0.9364,0.1247,0.0952
4,190083.5626,181205707024.2294,425682.6365,0.9198,0.1285,0.0958
5,183768.7645,130916661743.6878,361824.0757,0.9492,0.1264,0.0949
6,192314.8197,151066279021.5462,388672.4572,0.9487,0.1258,0.0957
7,196045.2490,158379219197.4160,397968.8671,0.9385,0.1289,0.0976
8,196717.9141,189091543659.9127,434846.5749,0.9292,0.1332,0.0981


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,170446.0156,207524593664.0000,455548.6875,0.9271,0.1073,0.0761
1,160728.9062,144275013632.0000,379835.5000,0.9466,0.1046,0.0744
2,170274.6562,224625000448.0000,473946.1875,0.9254,0.1093,0.0743
3,164277.0469,144714399744.0000,380413.4688,0.9487,0.1027,0.0741
4,159820.7031,174770307072.0000,418055.3750,0.9227,0.1043,0.0744
5,156446.3281,114346614784.0000,338151.7500,0.9557,0.1065,0.0749
6,164380.7969,132803936256.0000,364422.7500,0.9549,0.1037,0.0753
7,166388.5938,150291054592.0000,387673.9062,0.9416,0.1064,0.0756
8,170602.4219,181417541632.0000,425931.3750,0.9321,0.1142,0.0778


...

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,187917.0450,221928217924.5422,471092.5789,0.9220,0.1174,0.0862
1,176915.0753,147379665926.8586,383900.5938,0.9454,0.1157,0.0858
2,181564.5634,234204519536.5110,483946.8148,0.9222,0.1195,0.0833
3,180133.9271,219916392398.8154,468952.4415,0.9221,0.1127,0.0831
4,177173.1875,161466233144.1518,401828.6117,0.9286,0.1144,0.0841
5,175555.1252,157148732550.5052,396419.8942,0.9391,0.1184,0.0849
6,182375.8239,147082663999.1856,383513.5773,0.9500,0.1165,0.0860
7,183934.8213,160614684952.4900,400767.6196,0.9376,0.1188,0.0867
8,184806.2291,190267971026.3935,436197.1699,0.9287,0.1243,0.0878


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
bagged_catboost = ensemble_model(catboost, method = 'Bagging')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,174583.0193,210600668534.5772,458912.4846,0.9260,0.1116,0.0806
1,167942.2464,149495130793.9588,386646.0019,0.9447,0.1089,0.0798
2,170519.5692,204494109573.0965,452210.2493,0.9321,0.1118,0.0782
3,167981.2767,171041933638.7794,413572.1626,0.9394,0.1061,0.0778
4,165161.1003,149888904159.9075,387154.8839,0.9337,0.1087,0.0797
5,158070.2659,106450366368.1871,326267.3235,0.9587,0.1072,0.0779
6,167991.6525,122859951589.8624,350513.8394,0.9583,0.1079,0.0801
7,171777.7078,135182336916.2988,367671.5068,0.9475,0.1101,0.0809
8,171471.1116,164305412023.5836,405346.0399,0.9385,0.1161,0.0815


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
bagged_tuned_xgboost = ensemble_model(tuned_xgboost, method = "Bagging")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,166872.0000,209460969472.0000,457669.0625,0.9264,0.1044,0.0734
1,156739.3750,145367367680.0000,381270.7188,0.9462,0.1003,0.0716
2,160582.8125,210449317888.0000,458747.5625,0.9301,0.1053,0.0708
3,157239.2969,162617065472.0000,403258.0625,0.9424,0.0983,0.0702
4,152390.0938,152997445632.0000,391148.8750,0.9323,0.0997,0.0706
5,149469.9531,112093855744.0000,334804.2188,0.9565,0.1002,0.0706
6,157560.7656,134650888192.0000,366948.0625,0.9543,0.0997,0.0715
7,157701.6719,148688519168.0000,385601.5000,0.9422,0.1021,0.0718
8,159623.0000,158731960320.0000,398411.8125,0.9406,0.1090,0.0734


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
bagged_lightgbm = ensemble_model(lightgbm, method = "Bagging")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,195132.2741,224957477246.3822,474296.8240,0.9210,0.1263,0.0950
1,187506.9639,163152356364.1508,403921.2254,0.9396,0.1252,0.0940
2,191389.4978,236633157753.0982,486449.5429,0.9214,0.1276,0.0923
3,189983.7629,188256021248.0436,433884.8018,0.9333,0.1236,0.0941
4,186462.1358,170510113588.3334,412928.7028,0.9246,0.1256,0.0939
5,181981.9602,126441987284.7609,355586.8210,0.9510,0.1244,0.0937
6,191776.4970,158336127030.5292,397914.7233,0.9462,0.1256,0.0951
7,193132.2076,160403665097.2796,400504.2635,0.9377,0.1272,0.0961
8,192378.3879,182848621950.4250,427608.0237,0.9315,0.1310,0.0964


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
boosting_catboost = ensemble_model(catboost, method = "Boosting")
boosting_tuned_xgboost = ensemble_model(tuned_xgboost, method = "Boosting")
boosting_lightgbm = ensemble_model(lightgbm, method = "Boosting")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,178529.6774,193377989916.8174,439747.6434,0.9321,0.1144,0.0846
1,171457.5808,138056244623.1762,371559.2074,0.9489,0.1117,0.0840
2,173824.5440,189499826331.6330,435315.7777,0.9371,0.1149,0.0824
3,169743.7859,143650264911.2971,379012.2226,0.9491,0.1086,0.0820
4,169626.0334,158636464090.6366,398291.9332,0.9298,0.1115,0.0837
5,164646.7379,117123427013.5606,342233.0011,0.9546,0.1110,0.0824
6,171214.5200,121059736586.6784,347936.3973,0.9589,0.1113,0.0847
7,173753.3766,130115619266.3151,360715.4270,0.9494,0.1134,0.0846
8,174445.5968,160183874870.0471,400229.7776,0.9400,0.1189,0.0856


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,162521.4219,189470572544.0000,435282.1875,0.9334,0.1019,0.0721
1,156049.2500,148857323520.0000,385820.3125,0.9449,0.0991,0.0710
2,160562.9375,221111435264.0000,470224.8750,0.9266,0.1039,0.0702
3,157034.9062,165390598144.0000,406682.4375,0.9414,0.0971,0.0698
4,152144.5156,163822403584.0000,404749.8125,0.9275,0.0993,0.0703
5,149570.7656,112820969472.0000,335888.3438,0.9563,0.0997,0.0703
6,155703.9531,127021957120.0000,356401.4062,0.9568,0.0983,0.0711
7,153311.3438,128875806720.0000,358992.7812,0.9499,0.1001,0.0707
8,159609.4375,160099729408.0000,400124.6562,0.9400,0.1072,0.0729


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,203324.9249,193118002272.5116,439451.9340,0.9322,0.1333,0.1048
1,198627.9703,155802799266.5214,394718.6330,0.9423,0.1320,0.1033
2,200142.7957,203420056364.1513,451021.1263,0.9324,0.1345,0.1022
3,197720.2474,158043057135.6343,397546.2956,0.9440,0.1321,0.1030
4,194282.8989,182167121662.6377,426810.4048,0.9194,0.1322,0.1018
5,191122.2914,127352570118.7818,356864.9186,0.9506,0.1328,0.1029
6,198769.4924,144869359448.6486,380617.0772,0.9508,0.1341,0.1050
7,198495.5730,142073859490.7576,376926.8623,0.9448,0.1344,0.1043
8,200711.1672,176588044144.8353,420223.8024,0.9339,0.1372,0.1045


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
et = create_model("et")
rf = create_model("rf")
boosting_et = ensemble_model(et, method = "Boosting")
boosting_rf = ensemble_model(rf, method = "Boosting")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,158996.3744,201189239822.3940,448541.2354,0.9293,0.1015,0.0694
1,151673.7151,142721370670.5925,377784.8206,0.9472,0.0994,0.0690
2,152318.3013,203145578774.6304,450716.7390,0.9325,0.1011,0.0664
3,152361.7510,172455068946.1061,415277.0990,0.9389,0.0969,0.0667
4,151118.1301,175108286760.2752,418459.4207,0.9225,0.0999,0.0686
5,144549.2190,115717177507.5704,340172.2762,0.9551,0.1002,0.0682
6,152218.9177,130262132611.8727,360918.4570,0.9557,0.0992,0.0694
7,152202.3936,141360820814.1202,375979.8144,0.9451,0.0996,0.0688
8,154936.6256,151505430603.4804,389236.9851,0.9433,0.1072,0.0712


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,173265.9071,238335439600.0136,488196.1077,0.9163,0.1107,0.0754
1,165387.1359,176683617121.4718,420337.5038,0.9346,0.1060,0.0745
2,172761.3532,275305004760.9278,524695.1541,0.9086,0.1135,0.0739
3,167312.6332,199568746757.0362,446731.1795,0.9293,0.1054,0.0733
4,165430.3000,201439255897.4436,448819.8479,0.9109,0.1089,0.0747
5,156545.2303,128522671586.8886,358500.5880,0.9502,0.1056,0.0732
6,170176.0250,171912116136.0736,414622.8601,0.9416,0.1080,0.0759
7,167339.6119,183373121149.8036,428220.8789,0.9287,0.1078,0.0744
8,168741.3455,179761908282.7060,423983.3821,0.9327,0.1157,0.0773


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,07:49:53
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,str


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

Choose 6 best algorithm result to be estimator in stacking regressor.

In [ ]:
stacker = stack_models(estimator_list=[et, rf, catboost, tuned_xgboost, lightgbm, bagged_catboost])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,154292.9282,182778106339.1147,427525.5622,0.9358,0.0995,0.0687
1,148279.9514,128668347644.7600,358703.7045,0.9524,0.0971,0.0681
2,154587.4930,198979350741.8752,446071.0154,0.9339,0.1024,0.0677
3,148775.3739,141043239876.1543,375557.2391,0.9500,0.0934,0.0659
4,148303.3482,168901501880.7522,410976.2790,0.9253,0.0967,0.0677
5,141704.0134,96670344846.5965,310918.5502,0.9625,0.0973,0.0676
6,147293.4553,110975314202.0903,333129.5757,0.9623,0.0948,0.0680
7,147936.7764,124493959750.7990,352837.0158,0.9516,0.0959,0.0670
8,153347.5471,145204859348.5030,381057.5539,0.9456,0.1066,0.0707


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

finalize the model (fit all the data)

In [ ]:
final_stacker = finalize_model(stacker)

In [ ]:
test.drop(["price"], axis = 1, inplace = True)

In [ ]:
pred_test = predict_model(stacker, data = test)

In [ ]:
save_model(stacker,'stacker_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('date_feature_extractor',
                  TransformerWrapper(include=['date'],
                                     transformer=ExtractDateTimeFeatures())),
                 ('numerical_imputer',
                  TransformerWrapper(include=['area', 'InterestRate',
                                              'RentIndex', 'Available', 'Vacant',
                                              'lat', 'lng'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(inc...
                                                              monotone_constraints=None,
                                                              n_estimators=290,
                                                              n_jobs=-1,
                                                              num_parallel_tree=None,
                                       

In [ ]:
pred_test

,area,district,typeOfArea,tenure,street,project,property_key,date,train,CPI,...,Executive Condominium,Semi-detached,Strata Detached,Strata Semi-detached,Strata Terrace,Terrace,CCR,OCR,RCR,prediction_label
32,249.0,4,1,0,COVE DRIVE,TURQUOISE,p-940dd9922,2023-01-01,0,109.892998,...,0,0,0,0,0,0,1,0,0,4.351370e+06
36,202.0,4,1,0,COVE DRIVE,TURQUOISE,p-72a9ac344,2023-02-01,0,110.959000,...,0,0,0,0,0,0,1,0,0,3.290323e+06
44,75.0,7,1,0,BEACH ROAD,CITY GATE,p-3bd67ad7e,2023-03-01,0,111.185997,...,0,0,0,0,0,0,0,0,1,1.498514e+06
68,84.0,7,1,0,BEACH ROAD,CITY GATE,p-0ba6a9779,2023-01-01,0,109.892998,...,0,0,0,0,0,0,0,0,1,1.647733e+06
79,84.0,7,1,0,BEACH ROAD,CITY GATE,p-98a2eb94e,2023-01-01,0,109.892998,...,0,0,0,0,0,0,0,0,1,1.683622e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65246,65.0,27,1,0,YISHUN CENTRAL 1,NORTH PARK RESIDENCES,p-88070a1d9,2023-02-01,0,110.959000,...,0,0,0,0,0,0,0,1,0,1.151461e+06
65247,65.0,27,1,0,YISHUN CENTRAL 1,NORTH PARK RESIDENCES,p-88070a1d9,2023-03-01,0,111.185997,...,0,0,0,0,0,0,0,1,0,1.155746e+06
65252,52.0,27,1,0,YISHUN CENTRAL 1,NORTH PARK RESIDENCES,p-8bd80748c,2023-03-01,0,111.185997,...,0,0,0,0,0,0,0,1,0,8.919497e+05
65258,80.0,27,1,0,YISHUN CENTRAL 1,NORTH PARK RESIDENCES,p-be2eb0456,2023-01-01,0,109.892998,...,0,0,0,0,0,0,0,1,0,1.222669e+06


In [ ]:
predDf = pd.DataFrame({"property_key" : pred_test["property_key"],
                       "contractDate" : pred_test["date"],})
predDf = predDf.assign(prediction = pred_test["prediction_label"])
predDf.to_csv("Submission.csv", index = False)